# The Age of the Recommedation

## How to use this Notebook

You can open it at https://colab.research.google.com

You require two files from the IMBD (Internet Movie Data Base) named *movies.csv* and *ratings.csv* that should be uploaded to Google Colab

## Import libraries

In [ ]:
import numpy as np
import pandas as pd 

from matplotlib import pyplot as plt 
import seaborn as sns

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline
plt.style.use('ggplot')
pd.options.display.max_rows = 999

## Import data

In [ ]:
file_dir = '/content/'

In [ ]:
ratings_ = pd.read_csv(file_dir + 'ratings.csv')

In [ ]:
ratings_.info()

In [ ]:
ratings_.head(10)

In [ ]:
ratings_.describe()

In [ ]:
sns.countplot(x='rating', data=ratings_)

In [ ]:
movies_ = pd.read_csv(file_dir + 'movies.csv')

In [ ]:
movies_.info()

In [ ]:
movies_.head().T

## Merge of ratings data with movies metadata
Get the movie title into ratings data

In [ ]:
rated_movies = ratings_.merge(movies_, left_on='movieId', right_on='movieId')[['userId', 'movieId','title', 'rating']]

In [ ]:
rated_movies.head()

## EDA - Exploratory Data Analysis

In [ ]:
mean_ratings = rated_movies.pivot_table(index='title', values='rating', aggfunc=[np.mean, len])

In [ ]:
mean_ratings.columns = ['mean_rating', 'ratings']

In [ ]:
mean_ratings.head()

In [ ]:
mean_ratings.describe()

In [ ]:
mean_ratings[mean_ratings['ratings'] > 50].sort_values(by='mean_rating', ascending=False).head(10)

In [ ]:
mean_ratings[mean_ratings.index.str.contains('Indiana Jones')]

In [ ]:
# pivot ratings into movie features
ratings_pivot = rated_movies.pivot_table(
    index='userId',
    columns='title',
    aggfunc=np.mean,
    values='rating'
).fillna(0)

In [ ]:
ratings_pivot.info()

In [ ]:
ratings_pivot.head(10).T.head(10)

## Calculate cosine similarity between users and between movies

In [ ]:
sparse_ratings = sp.sparse.csr_matrix(ratings_pivot.values)

In [ ]:
user_similarity = cosine_similarity(sparse_ratings)
item_similarity = cosine_similarity(sparse_ratings.T)

In [ ]:
user_similarity_df = pd.DataFrame(user_similarity, index = ratings_pivot.index, columns = ratings_pivot.index)
item_similarity_df = pd.DataFrame(item_similarity, index = ratings_pivot.columns, columns = ratings_pivot.columns)

In [ ]:
item_similarity_df.head()

In [ ]:
user_similarity_df.head()

## Explore results of cosine similarity filters

### First with movies (items)

This function will return the top 10 shows with the highest cosine similarity value

In [ ]:
def top_movies(name):
    count = 1
    print('Similar shows to {} include:\n'.format(name))
    for item in item_similarity_df.sort_values(by = name, ascending = False).index[1:11]:
        print('No. {}: {}'.format(count, item))
        count +=1  

Ask the function top_movies regarding one movie to get the 10 more similar

In [ ]:
top_movies('Star Wars: Episode V - The Empire Strikes Back (1980)')

In [ ]:
top_movies('[REC] (2007)')

In [ ]:
top_movies('xXx (2002)')

### Same approach but now with users, instead of movies

Top 10 users with highest cosine similarity from a user defined

In [ ]:
# This function will return the top 10 users with the highest similarity value 

def top_users(user):
    
    if user not in ratings_pivot.index:
        return('No data available on user {}'.format(user))
    
    print('Most Similar Users:\n')
    sim_values = user_similarity_df.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:11]
    sim_users = user_similarity_df.sort_values(by=user, ascending=False).index[1:11]
    zipped = zip(sim_users, sim_values)
    for user, sim in zipped:
        print('User #{0}, Similarity value: {1:.2f}'.format(user, sim)) 

In [ ]:
top_users(7)

In [ ]:
ratings_[ratings_['userId'] == 7].head(10)

In [ ]:
ratings_[ratings_['userId'] == 403].head(15)

## Another approach for recommenders - clustering models

### Dimension reduction with PCA

Data is too wide, so we require reducing the attributes - predictors with a known technique called Principal Component Analysis (aka PCA)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(ratings_pivot)
pca_samples = pca.transform(ratings_pivot)

PCA compares the reduction of attributes with the capacity of explaining the variance

More reduccion implies usually less explained variance... so we lose somehow "resolution"

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

In [ ]:
pca_df = pd.DataFrame(pca_samples[:,0:50])

In [ ]:
pca_df.head(20)

### Creation of the clustering model 
Used to segment and group movies or users by similarity, in this case, the Euclidian distance between users or movies

In [ ]:
from sklearn.cluster import KMeans

sse = {}

for k in range(1,20):
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(pca_df)
    sse[k] = kmeans.inertia_

plt.figure(figsize=(15,5))
plt.plot(list(sse.keys()), list(sse.values()))
plt.show()

After comparing differnt numbers of clusters, we decide to use 15 as per the chart above


In [ ]:
kmeans = KMeans(n_clusters = 15)
kmeans.fit(pca_df)
pca_df['label'] = kmeans.labels_

In [ ]:
sns.pairplot(pca_df, vars=[0,1,2], hue='label')

In [ ]:
ratings_pivot['label'] = pca_df['label']

In [ ]:
cluster_ = ratings_pivot[ratings_pivot['label'] == 1].drop('label', axis=1)

In [ ]:
cluster_.replace({0: np.nan}, inplace = True)

In [ ]:
cluster_.mean().sort_values(ascending=False).head(10)

A loop that shows the top10 movies for each cluster defined

In [ ]:
for i in range(15):
    cluster_ = ratings_pivot[ratings_pivot['label'] == i].drop('label', axis=1)
    cluster_.replace({0: np.nan}, inplace = True)
    print('TOP10 movies for cluster ', i)
    votes = cluster_.count()
    most_voted_movies = votes[votes > 5]
    print(cluster_[most_voted_movies.index].mean().sort_values(ascending=False).head(10))
    print('-------')